In [5]:
"""
    This script processes the validation response JSON file generated by validate_response.py.
    As a reminder, the generated JSON file is a list of dictionaries, where each dictionary is a response from a verbal autopsy record generated by a language model.
    Each record is run 10 times, and the results are aggregated into a single dictionary.
    This code will loads the JSON file, then groupby rowid, aggregate the number of times a similar ICD10 code and CGHR10 code is returned, 
    compile all results into a dataframe, and then export the results to CSV files.
"""


import pandas as pd

PARSED_DATA = "repeated_sampled_0308_parsed.csv"

df = pd.read_csv(PARSED_DATA)

In [6]:
# Remove any ICDs with decimals
df[['cause1_icd10', 'cause2_icd10', 'cause3_icd10', 'cause4_icd10', 'cause5_icd10']] = df[['cause1_icd10', 'cause2_icd10', 'cause3_icd10', 'cause4_icd10', 'cause5_icd10']].map(lambda x: x.split('.')[0] if pd.notnull(x) else x)



In [7]:
grouped_df = df.groupby('rowid')
same_cause_count_df = pd.DataFrame(grouped_df['cause1_icd10'].value_counts())
# blank_df = pd.DataFrame(columns=[x for x in range(1,11)])
blank_df = pd.DataFrame(index=same_cause_count_df.reset_index().rowid.unique(), columns=[x for x in range(1,11)])
dummy_df = pd.get_dummies(same_cause_count_df['count']).astype(int).groupby('rowid').sum()
# same_cause_count_df = blank_df.merge(dummy_df, how='outer', right_index=True)
same_cause_count_df = blank_df.combine_first(dummy_df)
# same_cause_count_df = blank_df.merge(dummy_df, how='outer').infer_objects().fillna(0)
same_cause_count_df = same_cause_count_df.rename(columns=lambda x: f'same_cause1_icd10_{x}x')
same_cause_icd10_colnames = same_cause_count_df.columns

binarized_sum = same_cause_count_df.sum()
nbinarized_sum = same_cause_count_df[same_cause_icd10_colnames].apply(lambda x: x.astype(bool)).sum()

# reduce all non-zero values to 1 and sum
# print("same_cause_count_df.sum() binarized:")
# print(binarized_sum)
# print("same_cause_count_df.sum() non-binarized:")
# print(nbinarized_sum)

print("Binarized and non-binarized sum (binarized reduces repeated counts of a rowid record to 1)")
display(pd.DataFrame({'binarized': binarized_sum, 'non-binarized': nbinarized_sum}))

print(f"Majority repeated similarity (0.0-1.0): {binarized_sum.iloc[-5:].sum()/len(df.rowid.unique())}")

Binarized and non-binarized sum (binarized reduces repeated counts of a rowid record to 1)


,binarized,non-binarized
same_cause1_icd10_1x,19,17
same_cause1_icd10_2x,11,10
same_cause1_icd10_3x,6,6
same_cause1_icd10_4x,6,5
same_cause1_icd10_5x,6,4
same_cause1_icd10_6x,2,2
same_cause1_icd10_7x,6,6
same_cause1_icd10_8x,7,7
same_cause1_icd10_9x,13,13
same_cause1_icd10_10x,66,66


Majority repeated similarity (0.0-1.0): 0.94


In [8]:
"""
    Objective: Add the aggregated cause1_icd10 to the end of the dataframe. This completes the ICD10 processing portion. 
    The final dataframe have the following columns:
    - same_cause1_icd10_1x      (binarized, indiciating if the cause1_cghr10 is repeated 1 times)
    - same_cause1_icd10_2x
    - same_cause1_icd10_3x
    - same_cause1_icd10_4x
    - same_cause1_icd10_5x
    - same_cause1_icd10_6x
    - same_cause1_icd10_7x
    - same_cause1_icd10_8x
    - same_cause1_icd10_9x
    - same_cause1_icd10_10x     (binarized, indicating if the cause1_cghr10 is repeated 10 times)
    - cause1_icd10              (dictionary, {ICD10_1: count, ICD10_2: count, ...})
"""
aggregated_cause1_icd10_rows = []

for name, group in grouped_df:
    # print(f"Row ID: {name}", group['cause1_icd10'].value_counts().to_dict())
    aggregated_cause1_icd10_rows.append([name, group['cause1_icd10'].value_counts().to_dict()])
    
combined_icd10_df = pd.DataFrame(aggregated_cause1_icd10_rows, columns=['rowid', 'cause1_icd10']).set_index('rowid')

# pd.set_option('display.max_rows', None)

# combined_df

same_cause_count_df.merge(combined_icd10_df, left_index=True, right_index=True)

,same_cause1_icd10_1x,same_cause1_icd10_2x,same_cause1_icd10_3x,same_cause1_icd10_4x,same_cause1_icd10_5x,same_cause1_icd10_6x,same_cause1_icd10_7x,same_cause1_icd10_8x,same_cause1_icd10_9x,same_cause1_icd10_10x,cause1_icd10
14000252,0,0,0,0,0,0,0,0,0,1,{'A09': 10}
14000286,0,0,0,0,0,0,0,0,0,1,{'G83': 10}
14000296,0,0,0,0,0,0,0,0,0,1,{'K35': 10}
14000405,0,0,1,0,0,0,1,0,0,0,"{'R50': 7, 'J18': 3}"
14000435,0,0,0,0,0,0,0,0,0,1,{'B54': 10}
...,...,...,...,...,...,...,...,...,...,...,...
24002738,0,0,0,0,0,0,0,0,0,1,{'B54': 10}
24002795,0,1,0,0,0,0,0,1,0,0,"{'E75': 8, 'A83': 2}"
24002976,0,0,0,0,0,0,0,0,0,1,{'G83': 10}
24003163,0,0,0,0,0,0,0,0,0,1,{'G40': 10}


In [9]:
"""
    Objective: calculate the number of times a cause1_icd10 is repeated for each rowid using CGHR10.    
"""

# load the mapping
icd10_to_cghr_mapping = pd.read_csv('../data_202402/icd10_cghr10_v1.csv')

icd10_to_cghr_mapping.sample(5)

,cghr10_age,cghr10_title,icd10_code,icd10_range
5363,neo,Other,L82,L10-L99
2866,child,"Epilepsy, leukaemia, and other noncommunicable...",H40,H11-H59
3473,child,Other,P29,P28-P29
3679,child,"Epilepsy, leukaemia, and other noncommunicable...",R76,R76-R77
3451,child,"Epilepsy, leukaemia, and other noncommunicable...",N98,N75-N99


In [10]:
"""
    Objective: Load a minimal set of columns from the original datasets of all age groups and rounds.
"""

# Get age group and round once again and merge everything together into one dataframe
path_prefix = "../data_202402/"
merged_all_df = pd.DataFrame()

rounds = ['rd1', 'rd2']
age_groups = ['adult', 'child', 'neo']

for r in rounds:
    for a in age_groups:
        
        questionnaire_df =  pd.read_csv(f"{path_prefix}healsl_{r}_{a}_v1.csv")
        age_df =            pd.read_csv(f"{path_prefix}healsl_{r}_{a}_age_v1.csv")
        narrative_df =      pd.read_csv(f"{path_prefix}healsl_{r}_{a}_narrative_v1.csv")

        narrative_df = narrative_df.rename(columns={'summary': 'open_narrative'})
        
        # Merge the dataframes
        narrative_only = narrative_df[['rowid','open_narrative']]
        sex_only = questionnaire_df[['rowid','sex_cod']]
        age_only = age_df[['rowid','age_value_death','age_unit_death']]
        
        merged_df = narrative_only.merge(sex_only, on='rowid').merge(age_only, on='rowid')

        # Fill in missing values with empty string
        merged_df['sex_cod'] = merged_df['sex_cod'].fillna('')
        
        merged_df['age_group'] = f"{a}"
        merged_df['rd_group'] = f"{r}"

        assert not merged_df.isnull().values.any(), "Execution halted: NaN values found in merged_df"

        print(f"round: {r.ljust(10)} age group: {a.ljust(10)} len: {str(merged_df.shape[0]).ljust(10)}")
        # print(f"Sample of merged_df {merged_df.shape}:")
        # display(merged_df.sample(5))
        
        merged_all_df = pd.concat([merged_all_df, merged_df])
        


C:\Users\andyl\AppData\Local\Temp\ipykernel_30032\854693008.py:15: DtypeWarning: Columns (2,105,205,206,215,216,316) have mixed types. Specify dtype option on import or set low_memory=False.
  questionnaire_df =  pd.read_csv(f"{path_prefix}healsl_{r}_{a}_v1.csv")
C:\Users\andyl\AppData\Local\Temp\ipykernel_30032\854693008.py:15: DtypeWarning: Columns (182,245) have mixed types. Specify dtype option on import or set low_memory=False.
  questionnaire_df =  pd.read_csv(f"{path_prefix}healsl_{r}_{a}_v1.csv")


round: rd1        age group: adult      len: 4987      
round: rd1        age group: child      len: 2998      
round: rd1        age group: neo        len: 585       
round: rd2        age group: adult      len: 2025      
round: rd2        age group: child      len: 1059      
round: rd2        age group: neo        len: 233       


In [12]:
# Save different age group dataframes as seperate elements in a dictionary
cghr_map_helper = {}
for group in icd10_to_cghr_mapping.cghr10_age.unique():
    cghr_map_helper[group] = icd10_to_cghr_mapping[icd10_to_cghr_mapping.cghr10_age == group].set_index('icd10_code')

# cghr_map_helper['neo'].loc['O36']

In [22]:
cghr_df = df.assign(
    # output_msg = df.output.apply(lambda x: x['choices'][0]['message']['content'])
    cause1_cghr10 = df.apply(lambda row: 
        'NA' if row.cause1_icd10 not in cghr_map_helper[row.age_group].index                  # if index is not in group, return NA
        else cghr_map_helper[row.age_group].loc[row.cause1_icd10]['cghr10_title']       # if index is in group, return the cghr10_title
        , axis=1)
)

In [26]:
cghr_df[cghr_df.cause1_cghr10 == "NA"]

,rowid,cause1_icd10,cause1_icd10_prob,cause2_icd10,cause2_icd10_prob,cause3_icd10,cause3_icd10_prob,cause4_icd10,cause4_icd10_prob,cause5_icd10,...,param_model,param_system_prompt,param_user_prompt,output_usage_completion_tokens,output_usage_prompt_tokens,output_msg,output_probs,age_group,round,cause1_cghr10
68,14006882,O36,0.638113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,gpt-3.5-turbo-0125,You are a physician with expertise in determin...,Determine the underlying cause of death and pr...,4,368,O36.4,"[{'icd': 'O36.4', 'icd_linprob_mean': 0.638113...",neo,rd1,NA
168,14006882,O36,0.638113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,gpt-3.5-turbo-0125,You are a physician with expertise in determin...,Determine the underlying cause of death and pr...,4,368,O36.4,"[{'icd': 'O36.4', 'icd_linprob_mean': 0.638113...",neo,rd1,NA
368,14006882,O36,0.638762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,gpt-3.5-turbo-0125,You are a physician with expertise in determin...,Determine the underlying cause of death and pr...,4,368,O36.4,"[{'icd': 'O36.4', 'icd_linprob_mean': 0.638762...",neo,rd1,NA
568,14006882,O36,0.638113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,gpt-3.5-turbo-0125,You are a physician with expertise in determin...,Determine the underlying cause of death and pr...,4,368,O36.4,"[{'icd': 'O36.4', 'icd_linprob_mean': 0.638113...",neo,rd1,NA
768,14006882,O36,0.638113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,gpt-3.5-turbo-0125,You are a physician with expertise in determin...,Determine the underlying cause of death and pr...,4,368,O36.4,"[{'icd': 'O36.4', 'icd_linprob_mean': 0.638113...",neo,rd1,NA


In [24]:
# df.assign(cause1_cghr10=df['cause1_icd10'].map(lambda x: icd10_to_cghr_mapping[icd10_to_cghr_mapping['icd10_code'] == x]['cghr10_title'].values[0]))
# df.apply(lambda x: icd10_to_cghr_mapping[(icd10_to_cghr_mapping['cghr10_age'] == x['age_group']) & (icd10_to_cghr_mapping['icd10_code'] == x['cause1_icd10'])]['cghr10_title'].values[0], axis=1)

In [117]:
"""
    Objective: Map an ICD10 code to its their CGHR10 code. 
    Similar ICD10 codes can have different conversions depending on age groups. Therefore we need to determine the age group before converting.
"""

# Same ICD10 codes can have different conversions depending on age groups. Therefore we need to determine the age group before converting.
df['age_group'] = df['rowid'].map(merged_all_df.set_index('rowid')['age_group'])

# convert cause1_icd10 to cghr10_title, match icd10 and age group
df['cause1_cghr10'] = df.apply(lambda x: icd10_to_cghr_mapping[(icd10_to_cghr_mapping['cghr10_age'] == x['age_group']) & (icd10_to_cghr_mapping['icd10_code'] == x['cause1_icd10'])]['cghr10_title'].values[0], axis=1)

df.rename(columns={'Unnamed: 0': 'u_rowid'}, inplace=True)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# increase the number of rows to display
pd.set_option('display.max_rows', 25)

df[['u_rowid', 'rowid', 'cause1_icd10', 'cause1_cghr10', 'age_group']].head(10)

In [ ]:
"""
    Objective: Aggregate the number of times CGHR10 code is repeated and indicate in their respective columns as 1x...10x using binarization.
"""

same_cause_cghr10_count_df = pd.DataFrame(grouped_df['cause1_cghr10'].value_counts())
same_cause_cghr10_count_df = pd.get_dummies(same_cause_cghr10_count_df['count']).astype(int).groupby('rowid').sum().rename(columns=lambda x: f'same_cause1_cghr10_{x}x')

display(same_cause_cghr10_count_df)

# same_cghr_cause_count_df.sum()

# Alternative view: reduce all non-zero values to 1
same_cause_cghr10_count_df[['same_cause1_cghr10_1x', 'same_cause1_cghr10_2x', 'same_cause1_cghr10_3x', 'same_cause1_cghr10_4x', 'same_cause1_cghr10_5x',
                        'same_cause1_cghr10_6x', 'same_cause1_cghr10_7x', 'same_cause1_cghr10_8x', 'same_cause1_cghr10_9x', 'same_cause1_cghr10_10x']].apply(lambda x: x.astype(bool)).sum()

In [ ]:
"""
    Objective: Add the aggregated cause1_cghr10 to the end of the dataframe. This completes the CGHR10 processing portion.
    The final dataframe have the following columns:
    - same_cause1_cghr10_1x     (binarized, indiciating if the cause1_cghr10 is repeated 1 times)
    - same_cause1_cghr10_2x
    - same_cause1_cghr10_3x
    - same_cause1_cghr10_4x
    - same_cause1_cghr10_5x
    - same_cause1_cghr10_6x
    - same_cause1_cghr10_7x
    - same_cause1_cghr10_8x
    - same_cause1_cghr10_9x
    - same_cause1_cghr10_10x    (binarized, indiciating if the cause1_cghr10 is repeated 10 times)
    - cause1_cghr10             (dictionary, {CGHR10_1: count, CGHR10_2: count, ...})
"""

aggregated_cghr10_rows = []

for name, group in grouped_df:
    # print(f"Row ID: {name}", group['cause1_icd10'].value_counts().to_dict())
    aggregated_cghr10_rows.append([name, group['cause1_cghr10'].value_counts().to_dict()])
    
combined_cghr10_df = pd.DataFrame(aggregated_cghr10_rows, columns=['rowid', 'cause1_cghr10']).set_index('rowid')

same_cause_cghr10_count_df.merge(combined_cghr10_df, left_index=True, right_index=True)

In [ ]:
display(same_cause_count_df.merge(same_cause_cghr10_count_df, left_index=True, right_index=True)[['same_cause1_icd10_10x', 'same_cause1_cghr10_10x']])

print(f"Non-matching rows: ", end='')
print((same_cause_count_df.merge(same_cause_cghr10_count_df, left_index=True, right_index=True)[['same_cause1_icd10_10x', 'same_cause1_cghr10_10x']].sum(axis=1) != 2).sum())

In [ ]:
"""
    Objective: Merge the previously processed ICD10 and CGHR10 dataframes into one, and add the age_group and round columns to the end of the dataframe.
    The final dataframe have the following columns:
    - {all columns from processed ICD10 and CGHR10}
    - age_group     (adult, child, neo)
    - round         (integer 1 or 2)
"""

# combine the two dataframes

# temp_df_2 = same_cause_count_df.merge(combined_df, left_index=True, right_index=True)

print("Merging previously processed ICD10 and CGHR10 dataframes into one, and add the age_group and round columns to the end of the dataframe.")
temp_df_1 = same_cause_count_df.merge(combined_icd10_df, left_index=True, right_index=True)
temp_df_2 = same_cause_cghr10_count_df.merge(combined_cghr10_df, left_index=True, right_index=True)
temp_df_3 = merged_all_df.set_index('rowid')[['age_group', 'rd_group']].rename(columns={'rd_group': 'round'})
temp_df_3['round'] = temp_df_3['round'].str.replace(r'\D', '', regex=True).astype(int)

pd.options.display.max_columns = None

output_df = temp_df_1.join([temp_df_2, temp_df_3], how='inner').rename(columns={'cause1_icd10': 'cause_icd10', 'cause1_cghr10': 'cause_cghr10'})
output_df.sample(5)

# temp_df_1

In [ ]:
"""
    Objective: Export the results to CSV files.
"""

try:
    print("Exporting to CSV files...")
    output_df[output_df['round'] == 1].to_csv('healsl_rd1_rapid_gpt3_10xreruns_v2a.csv')
    output_df[output_df['round'] == 2].to_csv('healsl_rd2_rapid_gpt3_10xreruns_v2a.csv')
except Exception as e:
    print(e)

In [ ]:
merged_all_df.set_index('rowid').loc[24002795]